In [20]:
import sys
sys.path.insert(0, '/home/ldoyle/packages')
import h5py
import numpy as np
from tqdm import tqdm
import matplotlib

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import pysindy as ps
from lr_ed import localreg
from datetime import datetime
from scipy.fft import fft, fftfreq,rfft2

import scipy.signal as signal
from obspy.signal import filter as obsfilt
import glob
from pyproj import Proj


### Load data and Adjust Units

In [21]:
file_list = glob.glob("/home/ldoyle/notebooks/rad_10_smooth_data/*")

In [22]:
n = 1.4682
L = 51.04762

b=0
for q in np.arange(2,7):
    if b==0:
        working_dat = np.load(file_list[q])
        b=1
    else:
        working_dat = np.hstack((working_dat, np.load(file_list[q])))

# working_dat = np.load(file_list[15])
working_dat = ((2*np.pi)/2**16)*((1550.12 * 1e-9)/(0.78 * 4 * np.pi * n * L))*working_dat#[:,::10]
working_dat = working_dat *10**6#/(np.std(working_dat))
working_dat = working_dat[:,::10].T
mean_remove = (working_dat - np.mean(working_dat,axis=0)).T

working_dat = mean_remove.reshape(working_dat.shape[1],working_dat.shape[0],1)

In [23]:
print(working_dat.shape)
print(np.amax(working_dat))
print(np.amin(working_dat))

(5000, 6000, 1)
1.9783986066197787
-1.948681685699635


### Construct library

In [6]:
dt=1. #/200
xs = np.arange(working_dat.shape[0]) #2.0419047*np.arange(working_dat.shape[1])
# time = dt*np.arange(working_dat.shape[0])

library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]
pde_lib = ps.PDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    derivative_order=4,
    spatial_grid=xs,
    include_bias=True,
    is_uniform=True,
#     periodic=True
)

u_dot = ps.SpectralDerivative(axis=1)._differentiate(working_dat, t=1) #ps.FiniteDifference(axis=1)._differentiate(working_dat, t=1)

In [7]:
thresh = 0.05
optimizer = ps.STLSQ(threshold=thresh, alpha=1e-5, max_iter=50)
model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
model.fit(working_dat,  x_dot=u_dot, t=dt,ensemble=True)


library_ensemble_coefs = model.coef_list
mean_library_ensemble = np.mean(library_ensemble_coefs, axis=0)
std_library_ensemble = np.std(library_ensemble_coefs, axis=0)

In [8]:
print("Ensemble STLSQ with Threshold "+str(thresh))
model.print()

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.640 x0_1 + -0.041 x0_11 + 30.989 x0_111 + -1.741 x0_1111 + -0.152 x0x0_111 + -0.058 x0x0x0_111 + 0.341 x0x0_1111 + -2.780 x0x0x0_1111


In [9]:
u_test = (((2*np.pi)/2**16)*((1550.12 * 1e-9)/(0.78 * 4 * np.pi * n * L))*np.load(file_list[19])*10**6)[:,::10]
u_test = u_test.reshape(u_test.shape[0],u_test.shape[1],1)
u_dot_stlsq = model.predict(u_test)
u_dot_test = ps.FiniteDifference(axis=1)._differentiate(u_test, t=1)


In [10]:
np.mean(np.abs(u_dot_test[:,:,0]-u_dot_stlsq[:,:,0]))/np.mean(np.abs(u_dot_test[:,:,0]))

AxesArray(1.25370503)

### Include depth

In [24]:
latlondepth = np.genfromtxt('south_DAS_latlondepth.txt')

relevant = latlondepth[6558:11558,3]

In [25]:
h_shaped = np.tile(relevant.reshape(-1,1),working_dat.shape[1])
data_2 = np.stack((working_dat[:,:,0],h_shaped),
         axis=-1)
# data_2 = np.transpose(np.asarray([working_dat,h_shaped]),[1,2,0])

In [26]:
data_2.shape

(5000, 6000, 2)

In [27]:
inputs_per_library = np.array([[0,1],[0,1],[0,1]])
tensor_array = [[1, 1, 0]]

### Construct library

In [28]:
dt=1. #/200
xs = np.arange(5000) #2.0419047*np.arange(working_dat.shape[1])
# time = dt*np.arange(working_dat.shape[0])

library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]
pde_lib = ps.PDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    derivative_order=4,
    spatial_grid=xs,
    include_bias=True,
    is_uniform=True,
#     periodic=True
)

In [29]:
poly_library = ps.PolynomialLibrary(include_bias=False)
fourier_library = ps.FourierLibrary()

In [30]:
generalized_library = ps.GeneralizedLibrary(
    [poly_library, fourier_library, pde_lib],
    tensor_array=tensor_array,
    inputs_per_library=inputs_per_library,
)

In [33]:
print(generalized_library.get_feature_names)

<bound method GeneralizedLibrary.get_feature_names of <pysindy.feature_library.generalized_library.GeneralizedLibrary object at 0x7fd2c02f05e0>>


In [18]:
u_dot = ps.FiniteDifference(axis=1)._differentiate(working_dat, t=1)
data_dot = np.stack((u_dot[:,:,0],h_shaped),axis=-1)

In [19]:
optimizer = ps.STLSQ(threshold=0.05,alpha=1e-5)#, normalize_columns=True)
model = ps.SINDy(feature_library=generalized_library, optimizer=optimizer)
model.fit(data_2, x_dot=data_dot)


KeyboardInterrupt: 

In [ ]:
model.print()

In [ ]:
u_test = (((2*np.pi)/2**16)*((1550.12 * 1e-9)/(0.78 * 4 * np.pi * n * L))*np.load(file_list[19])*10**6)[:,::10]
# u_test = u_test.reshape(u_test.shape[0],u_test.shape[1],1)
data_test = np.stack((u_test,h_shaped[:,:u_test.shape[1]]),
         axis=-1)
u_dot_stlsq = model.predict(data_test)
u_dot_test = ps.FiniteDifference(axis=1)._differentiate(u_test, t=1)


In [ ]:
np.mean(np.abs(u_dot_test-u_dot_stlsq[:,:,0]))/np.mean(np.abs(u_dot_test))

In [14]:
thresh = 0.01
optimizer = ps.EnsembleOptimizer(ps.STLSQ(threshold=thresh, alpha=1e-5, max_iter=50),bagging=True)
model = ps.SINDy(feature_library=generalized_library, optimizer=optimizer)
model.fit(data_2,  x_dot=data_dot,ensemble=True)


library_ensemble_coefs = model.coef_list
mean_library_ensemble = np.mean(library_ensemble_coefs, axis=0)
std_library_ensemble = np.std(library_ensemble_coefs, axis=0)

In [15]:
model.print()

(x0)' = 0.637 x0_1 + 36.124 x0_111 + -0.001 x1x0_11 + -0.064 x0x0_111 + 0.038 x0x1_111 + -0.576 x1x0_111
(x1)' = 1.000 x1


In [16]:
u_test = (((2*np.pi)/2**16)*((1550.12 * 1e-9)/(0.78 * 4 * np.pi * n * L))*np.load(file_list[19])*10**6)[:,::10]
# u_test = u_test.reshape(u_test.shape[0],u_test.shape[1],1)
data_test = np.stack((u_test,h_shaped[:,:u_test.shape[1]]),
         axis=-1)
u_dot_stlsq = model.predict(data_test)
u_dot_test = ps.FiniteDifference(axis=1)._differentiate(u_test, t=1)


In [17]:
np.mean(np.abs(u_dot_test-u_dot_stlsq[:,:,0]))/np.mean(np.abs(u_dot_test))

AxesArray(1.24857167)